# Arbitrage Order Generation Notebook

This notebook makes orders. The find_arbitrage method is where logic is being stored.

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import multiprocessing as mp

In [66]:
options : pd.DataFrame = pd.read_csv("data/cleaned_options_data.csv")
options["day"] = options["ts_recv"].apply(lambda x: x.split("T")[0])

In [67]:
start_date : datetime = datetime(2024, 1, 1)
end_date : datetime = datetime(2024, 3, 30)

In [68]:
current_date = start_date
arbitrage_cutoff = 0.75 # cents
orders = []

options : pd.DataFrame = pd.read_csv("data/cleaned_options_data.csv")
options["day"] = options["ts_recv"].apply(lambda x: x.split("T")[0])
options['day'] = options.loc[:, 'day'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

def find_arbitrage(instrument_data: pd.DataFrame):
    global orders
    instrument_data = instrument_data.sort_values(['bid_px_00'])
    best_bid = instrument_data[['bid_px_00', 'bid_sz_00', 'symbol', 'ts_recv']].reset_index(drop=True)
    instrument_data = instrument_data.sort_values(['ask_px_00'], ascending=False)
    best_ask = instrument_data[['ask_px_00', 'ask_sz_00', 'symbol', 'ts_recv']].reset_index(drop=True)
    
    # concaenate the two dfs horizontally
    concatenated_df = pd.concat([best_bid, best_ask], axis=1)
    bid_pointer = len(concatenated_df) - 1
    ask_pointer = len(concatenated_df) - 1
    
    # display(concatenated_df)
    
    while bid_pointer < len(concatenated_df) and ask_pointer >= 0:
        if concatenated_df.at[bid_pointer, 'bid_px_00'] < concatenated_df.at[ask_pointer, 'ask_px_00'] + arbitrage_cutoff:
            break
        else:
            bid_size = concatenated_df.at[bid_pointer, 'bid_sz_00']
            ask_size = concatenated_df.at[ask_pointer, 'ask_sz_00']
            order_size = min(bid_size, ask_size)
            curr_bid, curr_ask = bid_pointer, ask_pointer
            bid_time, ask_time = best_bid.at[curr_bid, 'ts_recv'], best_ask.at[curr_ask, 'ts_recv']
            
            # if ask_time > bid_time:
            #     bid_pointer += 1
            #     continue
            
            pnl = (concatenated_df.at[bid_pointer, 'bid_px_00'] - concatenated_df.at[ask_pointer, 'ask_px_00']) * min(order_size, 100) * 100
            if bid_size > ask_size:
                concatenated_df.at[bid_pointer, 'bid_sz_00'] -= order_size
                ask_pointer -= 1
            elif bid_size < ask_size:
                concatenated_df.at[ask_pointer, 'ask_sz_00'] -= order_size
                bid_pointer -= 1
            else:
                concatenated_df.at[bid_pointer, 'bid_sz_00'] -= order_size
                concatenated_df.at[ask_pointer, 'ask_sz_00'] -= order_size
                bid_pointer -= 1
                ask_pointer -= 1
                        
            if order_size > 0:
                order_size = min(order_size, 100)
                orders.append({
                    "datetime" : best_bid.at[curr_bid, 'ts_recv'],
                    "option_symbol" : best_bid.at[curr_bid, 'symbol'],
                    "action" : "S",
                    "order_size" : order_size,
                    "pnl" : pnl
                })
                orders.append({
                    "datetime" : best_ask.at[curr_ask, 'ts_recv'],
                    "option_symbol" : best_ask.at[curr_ask, 'symbol'],
                    "action" : "B",
                    "order_size" : order_size,
                    "pnl" : pnl
                })
                
                
    return None

while current_date <= end_date:
    current_data = options.loc[options['day'] == current_date, :]
    print(current_date, current_data.shape[0])
    if current_data.shape[0] == 0:
        current_date = current_date + pd.DateOffset(days=1)
        continue
    
    
    current_data = current_data.sort_values(['instrument_id', 'bid_px_00'])
    instrument_ids = current_data['instrument_id'].unique()
    
    for instrument_id in instrument_ids:
        instrument_data = current_data.loc[current_data['instrument_id'] == instrument_id, :]

        find_arbitrage(instrument_data) # global appends to orders
    
    current_date = current_date + pd.DateOffset(days=1)

2024-01-01 00:00:00 0
2024-01-02 00:00:00 22556
2024-01-03 00:00:00 21409
2024-01-04 00:00:00 16526
2024-01-05 00:00:00 20568
2024-01-06 00:00:00 0
2024-01-07 00:00:00 0
2024-01-08 00:00:00 24570
2024-01-09 00:00:00 21654
2024-01-10 00:00:00 21136
2024-01-11 00:00:00 21267
2024-01-12 00:00:00 26472
2024-01-13 00:00:00 0
2024-01-14 00:00:00 0
2024-01-15 00:00:00 0
2024-01-16 00:00:00 28235
2024-01-17 00:00:00 33837
2024-01-18 00:00:00 76227
2024-01-19 00:00:00 30587
2024-01-20 00:00:00 0
2024-01-21 00:00:00 0
2024-01-22 00:00:00 33193
2024-01-23 00:00:00 21540
2024-01-24 00:00:00 29408
2024-01-25 00:00:00 20749
2024-01-26 00:00:00 18689
2024-01-27 00:00:00 0
2024-01-28 00:00:00 0
2024-01-29 00:00:00 22688
2024-01-30 00:00:00 22602
2024-01-31 00:00:00 22457
2024-02-01 00:00:00 19617
2024-02-02 00:00:00 31585
2024-02-03 00:00:00 0
2024-02-04 00:00:00 0
2024-02-05 00:00:00 21382
2024-02-06 00:00:00 16891
2024-02-07 00:00:00 27586
2024-02-08 00:00:00 26153
2024-02-09 00:00:00 30997
2024-02-

In [69]:
df = pd.DataFrame(orders)

print(df)

                               datetime          option_symbol action  \
0        2024-01-02T17:03:28.037040044Z  SPX   241220C05400000      S   
1        2024-01-02T19:07:59.751930078Z  SPX   241220C05400000      B   
2        2024-01-02T17:02:32.675571693Z  SPX   241220C05400000      S   
3        2024-01-02T19:07:59.751930078Z  SPX   241220C05400000      B   
4        2024-01-02T17:02:32.675571693Z  SPX   241220C05400000      S   
...                                 ...                    ...    ...   
1357807  2024-03-28T15:56:30.374053857Z  SPX   240920P05260000      B   
1357808  2024-03-28T14:21:45.870481194Z  SPX   240920P05260000      S   
1357809  2024-03-28T15:33:03.237796939Z  SPX   240920P05260000      B   
1357810  2024-03-28T14:21:45.870481194Z  SPX   240920P05260000      S   
1357811  2024-03-28T15:00:17.854589530Z  SPX   240920P05260000      B   

         order_size      pnl  
0                38  14060.0  
1                38  14060.0  
2               108  38880.0  

In [70]:
df.to_csv("data/labeledorders.csv", index=False)

In [71]:
df

,datetime,option_symbol,action,order_size,pnl
0,2024-01-02T17:03:28.037040044Z,SPX 241220C05400000,S,38,14060.0
1,2024-01-02T19:07:59.751930078Z,SPX 241220C05400000,B,38,14060.0
2,2024-01-02T17:02:32.675571693Z,SPX 241220C05400000,S,108,38880.0
3,2024-01-02T19:07:59.751930078Z,SPX 241220C05400000,B,108,38880.0
4,2024-01-02T17:02:32.675571693Z,SPX 241220C05400000,S,38,13680.0
...,...,...,...,...,...
1357807,2024-03-28T15:56:30.374053857Z,SPX 240920P05260000,B,37,7030.0
1357808,2024-03-28T14:21:45.870481194Z,SPX 240920P05260000,S,13,2340.0
1357809,2024-03-28T15:33:03.237796939Z,SPX 240920P05260000,B,13,2340.0
1357810,2024-03-28T14:21:45.870481194Z,SPX 240920P05260000,S,21,2310.0


In [72]:
sorted_df = df.sort_values(['pnl', 'option_symbol'], ascending=False).head(1000)

In [73]:
# duplicates = sorted_df[sorted_df.duplicated(['datetime', 'option_symbol'], keep=False)].sort_values(['datetime', 'option_symbol'])
# df_without_duplicates = sorted_df.drop_duplicates(subset=['datetime', 'option_symbol'], keep=False)

# collapsed = duplicates.groupby(['datetime', 'option_symbol']).agg({
#     'action' : 'first',
#     'order_size' : 'sum',
#     'pnl' : 'first'
# }).reset_index()

# final_df = pd.concat([df_without_duplicates, collapsed], ignore_index=True)
# final_df

In [74]:
sorted_df.to_csv("data/labeledorders.csv", index=False)